# Data Generation

This notebook contains the code to generate some of the files used by our project website. 

Data is taken from [OpenStats](https://openstats.altervista.org) and has to be pre-processed in the [Data Wrangling Notebook](data_wrangling.ipynb). This notebook assumes you have already pre-preprocessed the data. 

In [1]:
import os
import json
# Initial Spark
import pyspark
from pyspark.sql import SparkSession

from pyspark.sql.functions import explode
import pyspark.sql.types as T
from pyspark.sql.functions import lit, col, to_date

from pyspark.sql.functions import avg
from pyspark.sql.functions import stddev
from pyspark.sql.functions import count, countDistinct, concat, sum
from pyspark.sql.functions import percentile_approx
import pyspark.sql.functions as F

In [2]:
DATA_FOLDER = "..." # replace with the output folder of the data_wrangling script
file = "all.parquet"

SAVE_COMPUTATIONS = True

In [3]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("Trenitalia") \
    .config("spark.driver.memory", "16g") \
    .config("spark.executor.memory", "16g") \
    .getOrCreate()

sc = spark.sparkContext

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/01/23 22:25:29 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
sc._conf.getAll()

[('spark.driver.port', '64729'),
 ('spark.driver.host', '10.5.53.40'),
 ('spark.app.id', 'local-1706045130609'),
 ('spark.executor.id', 'driver'),
 ('spark.driver.memory', '16g'),
 ('spark.driver.extraJavaOptions',
  '-Djava.net.preferIPv6Addresses=false -XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.cs=ALL-UNNAMED --add-opens=java.base/sun.security.action=ALL-UNNAMED --add-opens=java.base/sun.util.calendar=ALL-UNNAMED --add-opens=java.security.jgss/sun.security.krb5=ALL-UNNAMED -Djdk.reflect.useDi

In [5]:
df = spark.read.parquet(os.path.join(DATA_FOLDER, "parquet", file))
print("Number of rows: {}".format(df.count()))

Number of rows: 32029349


In [6]:
df.columns

['train_arrival_stop_name',
 'train_class',
 'train_cn',
 'train_dl',
 'train_number',
 'train_arrival_time',
 'oae',
 'train_oaz',
 'train_od',
 'train_oo',
 'train_departure_time',
 'train_ope',
 'train_opz',
 'train_departure_stop_name',
 'train_pr',
 'train_arrival_delay',
 'train_departure_delay',
 'sea',
 'train_sep',
 'train_sub',
 'day',
 'month',
 'year',
 'date',
 '_id',
 'stop_name',
 'stop_arrival_time',
 'stop_departure_time',
 'stop_arrival_delay',
 'stop_departure_delay']

In [7]:
df = df.repartition(100, "stop_name", "train_class")

In [8]:
# load the dataset of the stops
stops = spark.read.csv("stops.csv", header=True, inferSchema=True)

stops_column_renamer = {
    "name": "stop_name",
    "lat": "stop_lat",
    "lon": "stop_lon",
    "station_id": "stop_id", 
    "name_short": "stop_name_short",
    "id_region": "stop_id_region",
}

for k, v in stops_column_renamer.items():
    stops = stops.withColumnRenamed(k, v)

In [9]:
# number of days
print("Number of days: ", df.select("date").distinct().count())

# first date
print("First date: ", df.select("date").distinct().orderBy("date").first().asDict()["date"])

# last date
print("Last date: ", df.select("date").distinct().orderBy("date", ascending=False).first().asDict()["date"])

# number of trains
print("Number of trains: ", df.select("train_number").distinct().count())

# number of stops
print("Number of stops: ", df.select("stop_name").distinct().count())

# number of train classes
print("Number of train classes: ", df.select("train_class").distinct().count())

# number of dataset rows
print("Number of dataset rows: ", df.count())


Number of days:  365


First date:  2023-01-01


Last date:  2023-12-31


Number of trains:  16011


Number of stops:  2426


Number of train classes:  11


Number of dataset rows:  32029349


In [10]:
print(df.columns)

['train_arrival_stop_name', 'train_class', 'train_cn', 'train_dl', 'train_number', 'train_arrival_time', 'oae', 'train_oaz', 'train_od', 'train_oo', 'train_departure_time', 'train_ope', 'train_opz', 'train_departure_stop_name', 'train_pr', 'train_arrival_delay', 'train_departure_delay', 'sea', 'train_sep', 'train_sub', 'day', 'month', 'year', 'date', '_id', 'stop_name', 'stop_arrival_time', 'stop_departure_time', 'stop_arrival_delay', 'stop_departure_delay']


# 0. Preprocessing
As step of preprocessing, we remove all delays that are anomalous, i.e. they are not in the range [-100, 300] minutes. 

In [11]:
interesting_columns = ['oae', 'train_oaz', "train_od", "train_oo", "train_cn", "train_dl", "train_ope", "train_opz", "train_pr", "sea", "train_sep", "train_sub" ]

# for each of the interesting_columns compute and show the distinct values
for c in interesting_columns:
    print("Distinct values for column {}: {}".format(c, df.select(c).distinct().count()))
    

Distinct values for column oae: 64


Distinct values for column train_oaz: 38247


Distinct values for column train_od: 391


Distinct values for column train_oo: 397


Distinct values for column train_cn: 976


...

Distinct values for column train_opz: 35793


Distinct values for column train_pr: 2


Distinct values for column sea: 13


Distinct values for column train_sep: 12


Distinct values for column train_sub: 4


In [12]:
# remove all values of delays that are not in the range [-100, 300] if they are numerical
MIN_DELAY = -100
MAX_DELAY = 300
df = df.filter((col("stop_arrival_delay").cast("double").isNull()) | (col("stop_arrival_delay").cast("double") >= MIN_DELAY) & (col("stop_arrival_delay").cast("double") <= MAX_DELAY))

# for trains that have a non-null `train_sub`, we replace `train_class` with `train_sub`
# NOTE: `train_sub` contains the category of high speed train (e.g. "FR", "FB", "FA" which stand for Frecciarossa, Frecciabianca, Frecciargento)
# When a train is high speed, `train_class` is empty, apart from one case, which is the FrecciaRossa Milano -> Paris where the train_class is EC (EuroCity)
# We replace that with FR (FrecciaRossa)
df = df.withColumn("train_class", F.when(col("train_sub").isNotNull(), col("train_sub")).otherwise(col("train_class")))

# remove all the trains that are not in the following classes
KEEP_TRAIN_CLASSES = ["IC", "ICN", "REG", "", "EC", "FA", "FB", "FR"]

df = df.filter(col("train_class").isin(KEEP_TRAIN_CLASSES))

# 1. Statistics for each station

For each distinct station, we want to obtain: 
1. Station name
2. Latitude, longitude
3. Average arrival delay
4. Median arrival delay
5. % of trains with delay > 3
6. % of trains with delay > 5
7. % of trains with delay > 10
8. Number of distinct train numbers that stopped 

In [13]:
# create a dataframe that counts the number of trains passed by each stop
stop_counts = df.groupBy("stop_name").agg(F.count("train_number").alias("count_stops")).cache()

In [14]:
# add column True if train had > 3 minutes of delay
data_stop = df.join(stops, on="stop_name", how="inner")

data_stop = data_stop \
    .filter(col("stop_arrival_delay").cast("double").isNotNull()) \
    .withColumn("stop_arrival_delay_double", col("stop_arrival_delay").cast("double")) \
    .drop("stop_arrival_delay") \
    .withColumnRenamed("stop_arrival_delay_double", "stop_arrival_delay") \
    .withColumn("3m_delay", col("stop_arrival_delay") > 3)\
    .withColumn("5m_delay", col("stop_arrival_delay") > 5)\
    .withColumn("10m_delay", col("stop_arrival_delay") > 10)\
    .withColumn("day_of_week", F.date_format(F.col("date"), "E"))\
    .withColumn("train_id", concat(col("train_class"), col("train_number")))

In [15]:
data_stop_stat = data_stop.groupBy("stop_name") \
    .agg(
        F.avg("stop_arrival_delay").alias("avg_arrival_delay"),
        F.percentile_approx("stop_arrival_delay", 0.5).alias("median_arrival_delay"),
        F.countDistinct("train_id").alias("count_trains"),
        # F.count("train_id").alias("count_stops"),
        F.sum(F.col("3m_delay").cast("long")).alias("count_3m_delay"),
        F.sum(F.col("5m_delay").cast("long")).alias("count_5m_delay"),
        F.sum(F.col("10m_delay").cast("long")).alias("count_10m_delay"),
        F.first("stop_lat").alias("stop_lat"),
        F.first("stop_lon").alias("stop_lon")
    ).join(stop_counts, on="stop_name", how="inner")

In [16]:
print("Number of stops: ", data_stop_stat.count())

Number of stops:  2136


In [17]:
print("Number of stops in (lat,long) dataset: ", stops.count())

Number of stops in (lat,long) dataset:  2961


In [18]:
print("Number of stops in final dataset: ", data_stop_stat.count())

Number of stops in final dataset:  2136


In [19]:
# we hardcode a minimum threshold, at least one train per month
MIN_THRESHOLD_DATES = 12 # at least for 12 different dates a train must have stopped here
MIN_THRESHOLD_STOP = 12 # at least 12 times a train must have stopped here
data_stop_stat = data_stop_stat.filter(col("count_stops") >= MIN_THRESHOLD_STOP)

# we filter out stops without lat/lon
data_stop_stat = data_stop_stat.filter(col("stop_lat").isNotNull() & col("stop_lon").isNotNull())

data_stop_stat = data_stop_stat.cache()

In [20]:
data_stop_pandas = data_stop_stat.toPandas()
data_stop_pandas.head(100)

,stop_name,avg_arrival_delay,median_arrival_delay,count_trains,count_3m_delay,count_5m_delay,count_10m_delay,stop_lat,stop_lon,count_stops
0,ZOAGLI,11.600000,10.0,29,31,28,18,44.335460,9.269126,13178
1,CALA SABINA,2.086808,2.0,20,210,42,12,41.011012,9.588100,1775
2,ROCCA D`EVANDRO,4.443245,2.0,34,2241,1440,689,41.438861,13.911338,6404
3,TERONTOLA CORTONA,5.615185,3.0,161,10524,6938,3108,43.210263,12.007906,25167
4,MONTIRONE,6.039469,3.0,30,4090,3065,1515,45.451352,10.239482,8330
...,...,...,...,...,...,...,...,...,...,...
95,BORGO FORNARI PER VOLTAGGIO,20.200000,16.0,10,9,9,7,44.588379,8.938295,10027
96,SAN MARCO D`ALUNZIO T.,3.005983,1.0,30,1675,874,255,38.094202,14.675909,6918
97,VEZZANO LIGURE,2.754894,2.0,107,4478,2681,827,44.127081,9.889280,15411
98,ZINASCO NUOVO,6.222046,5.0,34,5047,2920,790,45.124555,8.999657,7688


In [21]:
# save as csv
if SAVE_COMPUTATIONS:
    data_stop_pandas.to_csv(("dataset_generated/data_stop/data_stop.csv"), index=False)

### b. Statistics for each day of week

In [22]:
stop_counts = df.withColumn("day_of_week", F.date_format(F.col("date"), "E")).groupBy("stop_name", "day_of_week").agg(F.count("train_number").alias("count_stops")).cache()

data_stop_stat = data_stop.groupBy("stop_name", "day_of_week") \
    .agg(
        F.avg("stop_arrival_delay").alias("avg_arrival_delay"),
        F.percentile_approx("stop_arrival_delay", 0.5).alias("median_arrival_delay"),
        F.countDistinct("train_id").alias("count_trains"),
        # F.count("train_id").alias("count_stops"),
        F.sum(F.col("3m_delay").cast("long")).alias("count_3m_delay"),
        F.sum(F.col("5m_delay").cast("long")).alias("count_5m_delay"),
        F.sum(F.col("10m_delay").cast("long")).alias("count_10m_delay"),
        F.first("stop_lat").alias("stop_lat"),
        F.first("stop_lon").alias("stop_lon")
    ).join(stop_counts, on=["stop_name", "day_of_week"], how="inner")

In [23]:
data_stop_stat = data_stop_stat.filter(col("count_stops") >= MIN_THRESHOLD_STOP)
data_stop_stat_pandas = data_stop_stat.toPandas()

In [24]:
# create a file for each day of week with the statistics
if SAVE_COMPUTATIONS:
    for day in data_stop_stat_pandas["day_of_week"].unique():
        data_stop_stat_pandas[data_stop_stat_pandas["day_of_week"] == day].to_csv(("dataset_generated/data_stop/data_stop_{}.csv".format(day)), index=False)
    

### 1c. Statistics for each train type

In [25]:
stop_counts = df.groupBy("stop_name", "train_class").agg(
    F.count("train_number").alias("count_stops"),
    F.countDistinct("date").alias("count_dates")
)

In [26]:
data_stop_stat = data_stop.groupBy("stop_name", "train_class") \
    .agg(
        F.avg("stop_arrival_delay").alias("avg_arrival_delay"),
        F.percentile_approx("stop_arrival_delay", 0.5).alias("median_arrival_delay"),
        F.countDistinct("train_id").alias("count_trains"),
        F.count("train_id").alias("count_stops"),
        F.sum(F.col("3m_delay").cast("long")).alias("count_3m_delay"),
        F.sum(F.col("5m_delay").cast("long")).alias("count_5m_delay"),
        F.sum(F.col("10m_delay").cast("long")).alias("count_10m_delay"),
        F.first("stop_lat").alias("stop_lat"),
        F.first("stop_lon").alias("stop_lon")
    ).join(stop_counts, on=["stop_name", "train_class"], how="inner") \
    .filter(col("count_dates") >= MIN_THRESHOLD_DATES)

In [27]:
data_stop_stat_pandas = data_stop_stat.toPandas()

In [28]:
# create a file for each day of week with the statistics
if SAVE_COMPUTATIONS:
    for train_class in data_stop_stat_pandas["train_class"].unique():
        data_stop_stat_pandas[data_stop_stat_pandas["train_class"] == train_class].to_csv(("dataset_generated/data_stop/data_stop_class_{}.csv".format(train_class)), index=False)
    

### 1d. Statistics for each week day and train type


In [29]:
stop_counts = df.withColumn("day_of_week", F.date_format(F.col("date"), "E")).groupBy("stop_name", "train_class", "day_of_week").agg(
    F.count("train_number").alias("count_stops"),
    F.countDistinct("date").alias("count_dates")    
)

data_stop_stat = data_stop.groupBy("stop_name", "train_class", "day_of_week") \
    .agg(
        F.avg("stop_arrival_delay").alias("avg_arrival_delay"),
        F.percentile_approx("stop_arrival_delay", 0.5).alias("median_arrival_delay"),
        F.countDistinct("train_id").alias("count_trains"),
        # F.count("train_id").alias("count_stops"),
        F.sum(F.col("3m_delay").cast("long")).alias("count_3m_delay"),
        F.sum(F.col("5m_delay").cast("long")).alias("count_5m_delay"),
        F.sum(F.col("10m_delay").cast("long")).alias("count_10m_delay"),
        F.first("stop_lat").alias("stop_lat"),
        F.first("stop_lon").alias("stop_lon")
    ).join(stop_counts, on=["stop_name", "train_class", "day_of_week"], how="inner") \
    .filter(col("count_dates") >= MIN_THRESHOLD_DATES)

In [30]:
data_stop_stat_pandas = data_stop_stat.toPandas()

In [31]:
# for each combination of weekday and train_class, create a file with the statistics
if SAVE_COMPUTATIONS:
    for day in data_stop_stat_pandas["day_of_week"].unique():
        for train_class in data_stop_stat_pandas["train_class"].unique():
            data_stop_stat_pandas[(data_stop_stat_pandas["day_of_week"] == day) & (data_stop_stat_pandas["train_class"] == train_class)].to_csv(("dataset_generated/data_stop/data_stop_mix_{}_{}.csv".format(day, train_class)), index=False)


In [32]:
import shutil
if SAVE_COMPUTATIONS: 
    # zip the folder
    shutil.make_archive("dataset_generated/data_stop", 'zip', "dataset_generated/data_stop")

---
### 1d. Statistics for each region

Here we aim to extract some insights on train delays for each Italian region. Specifically, we want to compute the average arrival delay for each region, and the statistics used in the previous tasks. We are interested to see those differences for each train class. 

In [33]:
region_mapper = {
    1: "Lombardia",
    2: "Liguria",
    3: "Piemonte",
    4: "Valle d'Aosta",
    5: "Lazio",
    6: "Umbria",
    7: "Molise",
    8: "Emilia-Romagna",
    9: "Trentino-Alto Adige",
    10: "Friuli-Venezia Giulia",
    11: "Marche",
    12 : "Veneto",
    13: "Toscana",
    14: "Sicilia",
    15: "Basilicata",
    16: "Puglia",
    17: "Calabria",
    18: "Campania",
    19: "Abruzzo",
    20: "Sardegna",
    21: "Trentino-Alto Adige", #actually some stops of province of Trento.....
    22: "Trentino-Alto Adige", #actually some stops of province of Bolzano.....
    None: ""
}

In [34]:
from pyspark.sql.types import StringType
stops_region = stops.withColumn("stop_name_region", F.udf(lambda x: region_mapper[x], StringType())("stop_id_region"))

In [35]:
data_stop_first = df.join(stops_region, on="stop_name", how="inner")

data_stop = data_stop_first \
    .filter(col("stop_arrival_delay").cast("double").isNotNull()) \
    .withColumn("stop_arrival_delay_double", col("stop_arrival_delay").cast("double")) \
    .drop("stop_arrival_delay") \
    .withColumnRenamed("stop_arrival_delay_double", "stop_arrival_delay") \
    .withColumn("3m_delay", col("stop_arrival_delay") > 3)\
    .withColumn("5m_delay", col("stop_arrival_delay") > 5)\
    .withColumn("10m_delay", col("stop_arrival_delay") > 10)\
    .withColumn("day_of_week", F.date_format(F.col("date"), "E"))\
    .withColumn("train_id", concat(col("train_class"), col("train_number"))) \
    .cache()

24/01/23 22:32:05 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


##### For all trains

In [36]:
stop_counts = data_stop.groupBy("stop_name_region").agg(
    F.count("train_id").alias("count_stops"), 
    F.countDistinct("date").alias("count_dates")
)

data_stop_stat = data_stop.groupBy("stop_name_region") \
    .agg(
        F.avg("stop_arrival_delay").alias("avg_arrival_delay"),
        F.percentile_approx("stop_arrival_delay", 0.5).alias("median_arrival_delay"),
        F.countDistinct("train_id").alias("count_trains"),
        F.countDistinct("stop_name").alias("count_stations"),
        F.first("stop_id_region").alias("stop_id_region"),
    ).join(stop_counts, on=["stop_name_region"], how="inner") \
    .filter(col("count_dates") >= MIN_THRESHOLD_DATES) \
    .sort("stop_id_region")

In [37]:
if SAVE_COMPUTATIONS:
    data_stop_stat.toPandas().to_csv("dataset_generated/data_stop_region/data_stop_region.csv", index=False)

##### Per train class

In [38]:
stop_counts = data_stop.groupBy("stop_name_region", "train_class").agg(
    F.count("train_id").alias("count_stops"),
    F.countDistinct("date").alias("count_dates")
)

data_stop_stat = data_stop.groupBy("stop_name_region", "train_class") \
    .agg(
        F.avg("stop_arrival_delay").alias("avg_arrival_delay"),
        F.percentile_approx("stop_arrival_delay", 0.5).alias("median_arrival_delay"),
        F.countDistinct("train_id").alias("count_trains"),
        F.countDistinct("stop_name").alias("count_stations"),
        F.first("stop_id_region").alias("stop_id_region"),
    ).join(stop_counts, on=["stop_name_region", "train_class"], how="inner") \
    .filter(col("count_dates") >= MIN_THRESHOLD_DATES) \
    .sort("stop_id_region")

In [39]:
if SAVE_COMPUTATIONS:
    data_stop_stat_pandas = data_stop_stat.toPandas()

    for train_class in data_stop_stat_pandas["train_class"].unique():
        data_stop_stat_pandas[data_stop_stat_pandas["train_class"] == train_class].to_csv(("dataset_generated/data_stop_region/data_stop_region_class_{}.csv".format(train_class)), index=False)

In [40]:
# zip the folder
if SAVE_COMPUTATIONS:
    shutil.make_archive("dataset_generated/data_stop_region", 'zip', "dataset_generated/data_stop_region")

---

# 2. Train statistics
For each train, the goal is to store the following information:
1. Ordered list of the stations it stopped at
2. For each destination: 
    1. Average arrival delay
    2. Median arrival delay
    3. % of trains with delay > 3
    4. % of trains with delay > 5
    5. % of trains with delay > 10
    6. Number of days it stopped at the destination

To avoid keeping statistics for temporary stops and trains, we remove: 
- Trains that appeared in the dataset only <10  distinct times
- Stops for a train, that appeared less than 20% of the dates that the train appeared in the dataset

In [44]:
# first we add coordinates to each stop, so that we can drop stops for which we don't have the coordinates
df_trains = df.join(stops, on="stop_name", how="inner")

##### 1. For each train, we obtain the statistics of each of its stops, removing the stops that appear less than 20% of the dates that the train appeared in the dataset

Attention, we have to keep in mind that the same train number can refer to multiple trains, so we also have to group by the destination stop. 

In [45]:
# this shows that there's multiple trains with the same number
df \
    .filter((F.col("train_class") == "REG") & (F.col("train_number") == "4113"))\
    .select("train_arrival_stop_name")\
    .distinct()\
    .show()

+-----------------------+
|train_arrival_stop_name|
+-----------------------+
|           ROMA TERMINI|
|         COMO NORD LAGO|
+-----------------------+



In [46]:
# first we create a new column with the delay in minutes. For all stops this has to be the arrival_delay apart for the first stop of each train, for which it has to be the departure_delay
df_trains1 = df_trains.withColumn("delay", F.when(F.col("stop_arrival_delay") == "N", F.col("stop_departure_delay")).otherwise(F.col("stop_arrival_delay")))

# second, add a column "stop_arrival_time", which is the arrival time at the stop, in the format "HH:MM"
# if the stop is the first stop of the train, then the stop_arrival_time is the departure time of the train

df_trains1 = df_trains1\
    .withColumn("stop_time", F.when(F.col("stop_arrival_time") == 0, F.col("stop_departure_time")).otherwise(F.col("stop_arrival_time"))) \
    .withColumn("stop_time", F.date_format(F.col("stop_time").cast("timestamp"), "HH:mm"))

In [47]:
# convert the delay to double and add a counter if the train was 3m, 5m, 10m late
# add a column with the number of distinct dates for each train
# remove the column if the delay cannot be casted to double
df_trains2 = df_trains1 \
    .withColumn("delay", F.col("delay").cast("double")) \
    .filter(F.col("delay").isNotNull()) \
    .withColumn("3m_delay", F.when(F.col("delay") >= 3, 1).otherwise(0)) \
    .withColumn("5m_delay", F.when(F.col("delay") >= 5, 1).otherwise(0)) \
    .withColumn("10m_delay", F.when(F.col("delay") >= 10, 1).otherwise(0))\
    .select("train_departure_stop_name", "train_arrival_stop_name", "train_class", "train_number", "date", "stop_id", "stop_time", "stop_name", "stop_name_short", "stop_lat", "stop_lon", "stop_id_region", "3m_delay", "5m_delay", "10m_delay", "delay")


In [48]:

# For each train, we obtain the statistics of each of its stops, removing the stops that appear less than 20% of the dates that the train appeared in the dataset
# add the stop incremental number
df_trains_stat1 = df_trains2.groupBy("train_class", "train_number", "train_departure_stop_name", "train_arrival_stop_name", "stop_name") \
    .agg(
        F.avg("delay").alias("avg_arrival_delay"),
        F.percentile_approx("delay", 0.5).alias("median_arrival_delay"),
        F.countDistinct("date").alias("count_dates_stop"),
        F.min("date").alias("first_date"),
        F.max("date").alias("last_date"),
        F.sum(F.col("3m_delay").cast("long")).alias("count_3m_delay"),
        F.sum(F.col("5m_delay").cast("long")).alias("count_5m_delay"),
        F.sum(F.col("10m_delay").cast("long")).alias("count_10m_delay"),
        F.first("stop_lat").alias("stop_lat"),
        F.first("stop_lon").alias("stop_lon"),
        F.mode("stop_time").alias("stop_time")
    )


In [57]:
# counts how many distinct dates a train apparead in the dataset
# and filter out trains that didn't appear at least 4 times a month (12 times in total)
MIN_THRESHOLD_TRAIN = 12

df_trains_counts = df_trains2.groupBy("train_class", "train_number", "train_departure_stop_name", "train_arrival_stop_name") \
    .agg(
        F.countDistinct("date").alias("count_dates_train")
    ) \
    .filter(F.col("count_dates_train") >= MIN_THRESHOLD_TRAIN)

In order to store the data we create a mapper from a (train_class, train_number, train_arrival_stop_name) to an id, and we store the information of that train on a file with the name of the id. To do that, we add a column which is a monotonic increasing id, and we use that as the id of that train. 

In [58]:
# df_trains_counts = df_trains_counts \
#    .withColumn("train_id", F.monotonically_increasing_id()) \
#    .cache()


# Update, instead of the monotonically increasing Id, we add a concatenation of the train keys


df_trains_counts = df_trains_counts \
   .withColumn("train_id", F.concat(F.col("train_class"), F.col("train_number"), F.col("train_departure_stop_name"), F.col("train_arrival_stop_name"))) \
   .cache()

In [59]:
# merge the two datasets and filter out stations that appear in less than 20% of the dates
df_trains_stat2 = df_trains_stat1.join(df_trains_counts, on=["train_class", "train_number", "train_arrival_stop_name", "train_departure_stop_name"], how="inner") \
    .filter(F.col("count_dates_stop") >= F.col("count_dates_train") * 0.2) 

In [60]:
from pyspark.sql.window import Window

# add the stop incremental number
df_trains_stat3 = df_trains_stat2 \
    .withColumn("stop_number", F.row_number().over(Window.partitionBy("train_id").orderBy("stop_time"))) \
    .sort("train_id", "stop_number") 

In [61]:
# remove trains without class and order by "count_dates_train"
df_trains_stat3 = df_trains_stat3.filter(F.col("train_class")!= "")
df_trains_stat3 = df_trains_stat3.cache()

In [70]:
# now we store df_trains_stat2 in a csv file, partitioned by the column "train_id". 
# Each "train_id" should have a single file, and each file should contain the statistics of all the stops of the train

# remove folder if it already exists

if SAVE_COMPUTATIONS : 
    import shutil
    # if os.path.exists("dataset_generated/data_train_stat"): 
    shutil.rmtree("dataset_generated/data_train_stat")


    df_trains_stat3 \
        .repartition("train_id") \
        .write \
        .partitionBy("train_id") \
        .option("header", "true") \
        .csv("dataset_generated/data_train_stat/data_train_stat.csv")


In [71]:
SAVE_COMPUTATIONS = True

In [72]:
# the previous cell generates a folder with a lot of subfolders, one for each train. 
# each folder name is called "train_id=XXXXX", where XXXXX is the id of the train
# we want to generate a new folder with a single file for each train, where the file name is the train id
# the file should contain the statistics of all the stops of the train

# remove folder if it already exists
if SAVE_COMPUTATIONS :
    import shutil

    import os

    if os.path.exists("dataset_generated/data_train_stat_single_file") :
        shutil.rmtree("dataset_generated/data_train_stat_single_file")

    os.mkdir("dataset_generated/data_train_stat_single_file")

    import glob
    import shutil

    # get all the subfolders
    subfolders = [f.path for f in os.scandir("dataset_generated/data_train_stat/data_train_stat.csv") if f.is_dir() ] 

    # for each subfolder, get the csv file and rename it
    for subfolder in subfolders : 
        # get the csv file
        csv_file = glob.glob(subfolder + "/*.csv")[0]

        # get the train id
        train_id = subfolder.split("=")[1]

        # rename the file
        shutil.copy(csv_file, "dataset_generated/data_train_stat_single_file/" + train_id + ".csv")

In [73]:
# now we zip the folder data_train_stat_single_file and we put the zip in data_train_stat
if SAVE_COMPUTATIONS :
    import shutil
    shutil.make_archive("dataset_generated/data_train_stat/data_train_stat", 'zip', "dataset_generated/data_train_stat_single_file")

In [74]:
# read from file
# df_trains_stat3 = spark.read.option("header", "true").csv("dataset_generated/data_train_stat/data_train_stat.csv")

Now, we want to elaborate some general stastistics about a train, and store a dataset which is an index of the train ids, and the statistics of that train.

The aggregated statistics that we want to store are: 
1. Average arrival delay at each destination
2. Median arrival delay at each destination
3. % of trains with delay > 3 at each destination
4. % of trains with delay > 5 at each destination
5. % of trains with delay > 10 at each destination
6. Number of days the train ran
7. Number of stops of the train


In [ ]:
df_trains_stat3.show(1)

In [75]:
# Now, we want to elaborate some general stastistics about a train, and store a dataset which is an index of the train ids, and the statistics of that train.

# The aggregated statistics that we want to store are: 
# 1. Average arrival delay at each destination
# 2. Median arrival delay at each destination
# 3. % of trains with delay > 3 at each destination
# 4. % of trains with delay > 5 at each destination
# 5. % of trains with delay > 10 at each destination
# 6. Number of days the train ran
# 7. Number of stops of the train


df_trains_stat4 = df_trains_stat3 \
    .groupBy("train_id", "train_class", "train_number", "train_departure_stop_name", "train_arrival_stop_name") \
    .agg(
        F.avg("avg_arrival_delay").alias("avg_arrival_delay"),
        F.avg("median_arrival_delay").alias("median_arrival_delay"),
        (F.avg("count_3m_delay") / F.first("count_dates_train")).alias("perc_3m_delay"),
        (F.avg("count_5m_delay") / F.first("count_dates_train")).alias("perc_5m_delay"),
        (F.avg("count_10m_delay") / F.first("count_dates_train")).alias("perc_10m_delay"),
        F.first("count_dates_train").alias("count_dates_train"),
        F.countDistinct("stop_name").alias("count_stops_train"),
        F.first("first_date").alias("first_date"),
        F.first("last_date").alias("last_date"),
        F.min("stop_time").alias("departure_time"),
        F.max("stop_time").alias("arrival_time"),
    ) \
    .cache()

In [76]:
df_trains_stat4_pandas = df_trains_stat4.toPandas()

In [77]:
df_trains_stat4_pandas.head(10)

,train_id,train_class,train_number,train_departure_stop_name,train_arrival_stop_name,avg_arrival_delay,median_arrival_delay,perc_3m_delay,perc_5m_delay,perc_10m_delay,count_dates_train,count_stops_train,first_date,last_date,departure_time,arrival_time
0,FR9527MILANO CENTRALENAPOLI CENTRALE,FR,9527,MILANO CENTRALE,NAPOLI CENTRALE,17.491758,11.125000,0.714286,0.571429,0.410714,14,8,2023-08-07,2023-10-23,10:15,15:12
1,REG10021MI.P.GENOVAMORTARA,REG,10021,MI.P.GENOVA,MORTARA,5.922711,2.571429,0.486111,0.354663,0.205853,288,7,2023-01-02,2023-12-29,06:42,07:35
2,REG1139M N CADORNACOMO NORD LAGO,REG,1139,M N CADORNA,COMO NORD LAGO,0.236287,0.000000,0.037975,0.012658,0.000000,237,1,2023-01-02,2023-12-23,12:32,12:32
3,REG11818AOSTAIVREA,REG,11818,AOSTA,IVREA,2.311655,1.625000,0.382601,0.178632,0.031672,296,8,2023-01-02,2023-12-30,07:04,08:22
4,REG1224NOVARA NORDM N CADORNA,REG,1224,NOVARA NORD,M N CADORNA,0.811966,0.000000,0.113248,0.068376,0.029915,234,4,2023-01-02,2023-12-23,08:08,08:22
5,REG12683ROMA TERMINILATINA,REG,12683,ROMA TERMINI,LATINA,2.504683,0.833333,0.283977,0.166667,0.078684,233,6,2023-01-02,2023-12-28,17:06,17:52
6,REG12917MESSINA CENTRALEPATTI-SAN PIERO PATTI,REG,12917,MESSINA CENTRALE,PATTI-SAN PIERO PATTI,2.108881,1.444444,0.356557,0.154827,0.018670,244,9,2023-01-02,2023-12-29,21:23,22:28
7,REG12991MESSINA CENTRALECATANIA AER. FONTANAROSSA,REG,12991,MESSINA CENTRALE,CATANIA AER. FONTANAROSSA,1.888889,1.055556,0.307692,0.192308,0.029915,13,18,2023-12-12,2023-12-29,19:27,21:26
8,REG16020VENEZIA SANTA LUCIATRIESTE CENTRALE,REG,16020,VENEZIA SANTA LUCIA,TRIESTE CENTRALE,1.934426,0.863636,0.239195,0.116990,0.039493,61,22,2023-01-01,2023-12-31,19:01,22:44
9,REG16707BOLZANOTRENTO,REG,16707,BOLZANO,TRENTO,5.401639,4.000000,0.704918,0.385246,0.114754,61,2,2023-01-01,2023-12-31,19:41,19:54


In [78]:
if SAVE_COMPUTATIONS :    
    # store the file in a csv file  
    df_trains_stat4_pandas.to_csv("dataset_generated/data_train_index.csv", index=False)

    # zip the file
    !zip dataset_generated/data_train_index.zip dataset_generated/data_train_index.csv

  adding: dataset_generated/data_train_index.csv (deflated 68%)


Now we store the data in a better format, which is a folder in which each file is a train, and the filename is the train id. 

In [79]:
import os
import shutil
# if the folder already exists, delete it
if os.path.exists("renamed_csv_files"):
    shutil.rmtree("renamed_csv_files")

# create a new directory to store the renamed CSV files
if not os.path.exists("renamed_csv_files"):
    os.mkdir("renamed_csv_files")

# loop through all directories in the "data_train_stat/data_train_stat.csv" directory that start with "train_id="
for dirpath, dirnames, filenames in os.walk("dataset_generated/data_train_stat/data_train_stat.csv"):
    for dirname in dirnames:
        if dirname.startswith("train_id="):
            # extract the ID from the directory name
            id = dirname.split("=")[1]
            # loop through all CSV files in the directory
            for filename in os.listdir(os.path.join(dirpath, dirname)):
                if filename.endswith(".csv"):
                    # rename the file to "[ID].csv" and move it to the "renamed_csv_files" directory
                    src_path = os.path.join(dirpath, dirname, filename)
                    dst_path = os.path.join("renamed_csv_files", f"{id}.csv")
                    shutil.copy(src_path, dst_path)


# create a zip file containing the "renamed_csv_files" directory, the files have to be in a directory when unzipped
shutil.make_archive("renamed_csv_files", "zip", "renamed_csv_files")

# delete the "renamed_csv_files" directory
shutil.rmtree("renamed_csv_files")

# 3. Path of trains over a map
Now that we have computed a dataset with the statistics for each train, in which we managed to extract the timetable, we can use it to plot the path of each train on a map. 

The task of matching information of a train, with its exact journey on the railway is a complex task. The algorithm developed by [Bast and Brosi (2019)](https://ad-publications.cs.uni-freiburg.de/SIGSPATIAL_Sparse%20map%20matching%202018.pdf) matches the GTFS schedule of a train with the OpenStreetMap railway network. Thankfully, they published the code of this algorithm on GitHub. Their library takes a GTFS schedule of a train and a railway network in OpenStreetMap format and returns the most likely journey of the train on the railway network in `shapefile` format. GTFS is a standard format for public transport schedules, promoted by Google.

First, we need to generate a GTFS timetable for our trains. Then, we can run the algorithm a produce a shapefile for each train.

In [80]:
trains_timetable = df_trains_stat3 \
    .select("train_id", "train_class", "train_number", "train_departure_stop_name", "train_arrival_stop_name", "stop_number", "stop_name", "stop_time", "stop_lat", "stop_lon")

# need to join with the stops to get stop_id that we have lost on the way
trains_timetable = trains_timetable \
    .join(stops, on=["stop_name", "stop_lat", "stop_lon"], how="inner")

trains_timetable_pandas = trains_timetable.toPandas()

In [81]:
trains_timetable_pandas.head(10)

,stop_name,stop_lat,stop_lon,train_id,train_class,train_number,train_departure_stop_name,train_arrival_stop_name,stop_number,stop_time,stop_id,stop_name_short,stop_id_region
0,VENEZIA SANTA LUCIA,45.441569,12.320882,EC1280VENEZIA SANTA LUCIABRENNERO,EC,1280,VENEZIA SANTA LUCIA,BRENNERO,1,15:35,S02593,Venezia S.Lucia,12.0
1,VENEZIA MESTRE,45.482123,12.232069,EC1280VENEZIA SANTA LUCIABRENNERO,EC,1280,VENEZIA SANTA LUCIA,BRENNERO,2,15:45,S02589,Venezia Mestre,12.0
2,PADOVA,45.417846,11.880794,EC1280VENEZIA SANTA LUCIABRENNERO,EC,1280,VENEZIA SANTA LUCIA,BRENNERO,3,16:01,S02581,Padova,12.0
3,VICENZA,45.541102,11.540762,EC1280VENEZIA SANTA LUCIABRENNERO,EC,1280,VENEZIA SANTA LUCIA,BRENNERO,4,16:22,S02446,Vicenza,12.0
4,VERONA PORTA NUOVA,45.428603,10.982377,EC1280VENEZIA SANTA LUCIABRENNERO,EC,1280,VENEZIA SANTA LUCIA,BRENNERO,5,16:50,S02430,Verona P.Nuova,12.0
5,ROVERETO,45.890985,11.033518,EC1280VENEZIA SANTA LUCIABRENNERO,EC,1280,VENEZIA SANTA LUCIA,BRENNERO,6,17:41,S02044,Rovereto,21.0
6,TRENTO,46.072414,11.118941,EC1280VENEZIA SANTA LUCIABRENNERO,EC,1280,VENEZIA SANTA LUCIA,BRENNERO,7,17:57,S02038,Trento,21.0
7,BRESSANONE,46.710088,11.649677,EC1280VENEZIA SANTA LUCIABRENNERO,EC,1280,VENEZIA SANTA LUCIA,BRENNERO,8,19:02,S02014,Bressanone,9.0
8,BRENNERO,47.004117,11.505711,EC1280VENEZIA SANTA LUCIABRENNERO,EC,1280,VENEZIA SANTA LUCIA,BRENNERO,9,19:48,S02001,BRENNERO,22.0
9,BRENNERO,47.004117,11.505711,EC1281BRENNEROVENEZIA SANTA LUCIA,EC,1281,BRENNERO,VENEZIA SANTA LUCIA,1,10:10,S02001,BRENNERO,22.0


In [82]:
# look at distinct values of train_class
trains_timetable_pandas["train_class"].unique()

array(['EC', 'FA', 'FB', 'FR', 'IC', 'ICN', 'REG'], dtype=object)

In [83]:
import pandas as pd

# delete folder if it exists
import shutil
if os.path.exists("gtfs"):
    shutil.rmtree("gtfs")

# create folder gtfs 
if not os.path.exists("gtfs"):
    os.mkdir("gtfs")

# 1. Create the agency.txt file
agency = {
    "agency_id": "1",
    "agency_name": "Trenitalia",
    "agency_url": "https://www.trenitalia.com",
    "agency_timezone": "Europe/Rome",
    "agency_lang": "it",
    "agency_phone": ""
}
agency = pd.DataFrame(agency, index=[0])
agency.to_csv("gtfs/agency.txt", index=False)

In [84]:
# 2. create routes.txt
# docs: https://gtfs.org/schedule/reference/#routestxt


routes = pd.DataFrame()

# get unique tuples (train_id, train_class, train_number)
train_ids = trains_timetable_pandas[["train_id", "train_class", "train_number"]].drop_duplicates()

routes["route_id"] = train_ids["train_id"]
routes["route_short_name"] = train_ids["train_class"] + " " + train_ids["train_number"]
routes["route_long_name"] = train_ids["train_class"] + " " + train_ids["train_number"]

color_mapper = {
    "IC": "E0A434",
    "REG": "036864",
    "ICN": "E0A434",
    "AV": "DC263B",
    "EC": "DC263B",
    "FR": "DC263B",
    "FA": "DC263B",
    "FB": "DC263B",
}

routes["agency_id"] = "1"
routes["route_type"] = "2" # 2 is train

routes["route_color"] = routes["route_short_name"].apply(lambda x: color_mapper[x.split(" ")[0]])

routes.to_csv("gtfs/routes.txt", index=False)

In [85]:
# 3. create stops.txt
# docs: https://gtfs.org/schedule/reference/#stopstxt
stops_gtfs = trains_timetable_pandas[["stop_id", "stop_name", "stop_lat", "stop_lon"]].drop_duplicates()
stops_gtfs.to_csv("gtfs/stops.txt", index=False)

In [86]:
# 4. Tough one, create calendar.txt and trips.txt
# calendar describes the span of a service
# trips describes the service for a particular route

# 4a: create calendar.txt
# docs: https://gtfs.org/schedule/reference/#calendartxt
calendar = {
    # we just create a service that runs every day for the whole year
    "service_id": "1",
    "monday": "1",
    "tuesday": "1",
    "wednesday": "1",
    "thursday": "1",
    "friday": "1",
    "saturday": "1",
    "sunday": "1",
    "start_date": "19000101",
    "end_date": "21000101",
}
calendar = pd.DataFrame(calendar, index=[0])
calendar.to_csv("gtfs/calendar.txt", index=False)

In [87]:
# 4b: create trips.txt
# docs: https://gtfs.org/schedule/reference/#tripstxt
# trips = {
#     "route_id": "1",
#     "service_id": "1",
#     "trip_id": "1",
#     "trip_headsign": query_7["train_class"] + " " + query_7["train_number"],
#     "trip_short_name": query_7["train_class"] + " " + query_7["train_number"],
#     "direction_id": "",
#     "block_id": "",
#     "shape_id": "",
#     "wheelchair_accessible": "",
#     "bikes_allowed": "",
# }

# trips = pd.DataFrame(trips, index=[0])

trips = pd.DataFrame()

trips["route_id"] = train_ids["train_id"]
trips["service_id"] = "1"
trips["trip_id"] = train_ids["train_id"]
trips["trip_headsign"] = train_ids["train_class"] + " " + train_ids["train_number"]
trips["trip_short_name"] = train_ids["train_class"] + " " + train_ids["train_number"]
trips["direction_id"] = ""
trips["block_id"] = ""
trips["shape_id"] = ""
trips["wheelchair_accessible"] = ""
trips["bikes_allowed"] = ""


trips.to_csv("gtfs/trips.txt", index=False)

In [88]:
trains_timetable_pandas.head(10)

,stop_name,stop_lat,stop_lon,train_id,train_class,train_number,train_departure_stop_name,train_arrival_stop_name,stop_number,stop_time,stop_id,stop_name_short,stop_id_region
0,VENEZIA SANTA LUCIA,45.441569,12.320882,EC1280VENEZIA SANTA LUCIABRENNERO,EC,1280,VENEZIA SANTA LUCIA,BRENNERO,1,15:35,S02593,Venezia S.Lucia,12.0
1,VENEZIA MESTRE,45.482123,12.232069,EC1280VENEZIA SANTA LUCIABRENNERO,EC,1280,VENEZIA SANTA LUCIA,BRENNERO,2,15:45,S02589,Venezia Mestre,12.0
2,PADOVA,45.417846,11.880794,EC1280VENEZIA SANTA LUCIABRENNERO,EC,1280,VENEZIA SANTA LUCIA,BRENNERO,3,16:01,S02581,Padova,12.0
3,VICENZA,45.541102,11.540762,EC1280VENEZIA SANTA LUCIABRENNERO,EC,1280,VENEZIA SANTA LUCIA,BRENNERO,4,16:22,S02446,Vicenza,12.0
4,VERONA PORTA NUOVA,45.428603,10.982377,EC1280VENEZIA SANTA LUCIABRENNERO,EC,1280,VENEZIA SANTA LUCIA,BRENNERO,5,16:50,S02430,Verona P.Nuova,12.0
5,ROVERETO,45.890985,11.033518,EC1280VENEZIA SANTA LUCIABRENNERO,EC,1280,VENEZIA SANTA LUCIA,BRENNERO,6,17:41,S02044,Rovereto,21.0
6,TRENTO,46.072414,11.118941,EC1280VENEZIA SANTA LUCIABRENNERO,EC,1280,VENEZIA SANTA LUCIA,BRENNERO,7,17:57,S02038,Trento,21.0
7,BRESSANONE,46.710088,11.649677,EC1280VENEZIA SANTA LUCIABRENNERO,EC,1280,VENEZIA SANTA LUCIA,BRENNERO,8,19:02,S02014,Bressanone,9.0
8,BRENNERO,47.004117,11.505711,EC1280VENEZIA SANTA LUCIABRENNERO,EC,1280,VENEZIA SANTA LUCIA,BRENNERO,9,19:48,S02001,BRENNERO,22.0
9,BRENNERO,47.004117,11.505711,EC1281BRENNEROVENEZIA SANTA LUCIA,EC,1281,BRENNERO,VENEZIA SANTA LUCIA,1,10:10,S02001,BRENNERO,22.0


In [89]:
# 5. create stop_times.txt
# docs: https://gtfs.org/schedule/reference/#stop_timestxt
# convert stop_departure_time to HH:MM:SS using pyspark function

# stop_times: pd.DataFrame = train_data_day_df[["stop_id", "stop_departure_time", "stop_arrival_time"]] \
#     .rename(columns={"stop_id": "stop_id", "stop_departure_time": "departure_time", "stop_arrival_time": "arrival_time"})


# def convert_to_hh_mm_ss(time):
    
                                                                                                            
# # add column with stop_sequence
# stop_times["stop_sequence"] = stop_times.index + 1
# # add column with trip_id
# stop_times["trip_id"] = "1"
# # put departure time of the last stop equal to the arrival time
# stop_times.loc[stop_times.index[-1], "departure_time"] = stop_times.loc[stop_times.index[-1], "arrival_time"]
# store it

stop_times_gtfs = pd.DataFrame()

stop_times_gtfs["trip_id"] = trains_timetable_pandas["train_id"]
stop_times_gtfs["arrival_time"] = trains_timetable_pandas["stop_time"].apply(lambda x: x + ":00")
stop_times_gtfs["departure_time"] = trains_timetable_pandas["stop_time"].apply(lambda x: x + ":00")
stop_times_gtfs["stop_id"] = trains_timetable_pandas["stop_id"]
stop_times_gtfs["stop_sequence"] = trains_timetable_pandas["stop_number"]

stop_times_gtfs.to_csv("gtfs/stop_times.txt", index=False)

In [90]:
# generate a zip using libzip
import zipfile
zf = zipfile.ZipFile('gtfs.zip', mode='w')
try:
    zf.write("gtfs/agency.txt")
    zf.write("gtfs/calendar.txt")
    zf.write("gtfs/routes.txt")
    zf.write("gtfs/stops.txt")
    zf.write("gtfs/stop_times.txt")
    zf.write("gtfs/trips.txt")
finally:
    zf.close()

##### Run the algorithm
We downloaded and installed pfaedle as well as an OSM dump for Italy. 

In [ ]:
"""pfaedle -x ~/Downloads/italy-latest.osm {}""".format(os.path.join(os.getcwd(), "gtfs"))

Now that we have all the shapes in the "shapes.txt" file, we want to separate all those files into one file for each train.

In [94]:
import pandas as pd

shapes_gtfs = pd.read_csv("gtfs-out/shapes.txt")
trips_gtfs = pd.read_csv("gtfs-out/trips.txt")

shapes_gtfs.head(10)

,shape_id,shape_pt_lat,shape_pt_lon,shape_pt_sequence,shape_dist_traveled
0,shp_2_1,45.760090,10.996771,1,0.000
1,shp_2_1,45.760235,10.996886,2,18.672
2,shp_2_1,45.761852,10.998228,3,226.667
3,shp_2_1,45.762783,10.998978,4,345.598
4,shp_2_1,45.763184,10.999331,5,397.773
5,shp_2_1,45.763657,10.999795,6,461.719
6,shp_2_1,45.764145,11.000370,7,531.932
7,shp_2_1,45.764538,11.000874,8,590.560
8,shp_2_1,45.764877,11.001401,9,646.452
9,shp_2_1,45.765221,11.001960,10,704.023


In [95]:
trips_gtfs = trips_gtfs[["trip_id","shape_id"]]
# merge columns on shape_id
shapes_gtfs_merged = shapes_gtfs.merge(trips_gtfs, on="shape_id", how="inner")

In [96]:
# drop shape_id
print("The number of distinct trains is ", shapes_gtfs_merged["trip_id"].nunique(), "while the number of distinct shapes is ", shapes_gtfs_merged["shape_id"].nunique())

The number of distinct trains is  13828 while the number of distinct shapes is  3639


We could save some space by storing the shapes independently from the trains, since there's multiple trains that share the same shape. However, to keep the frontend of the website simple, we decided to store the shape of each train in a separated file. 

In [97]:
pd.unique(shapes_gtfs_merged["trip_id"])

array(['REG16144ALATRENTO', 'REG16148ALATRENTO', 'REG16150ALATRENTO', ...,
       'FR9805TORINO P.NUOVALECCE',
       'ICN36136REGGIO CALABRIA CENTRALETORINO P.NUOVA',
       'ICN36140REGGIO CALABRIA CENTRALETORINO P.NUOVA'], dtype=object)

In [108]:
from tqdm import tqdm

# delete folder if exists
import shutil
if os.path.exists("dataset_generated/trains_shapes"):
    shutil.rmtree("dataset_generated/trains_shapes")

# create folder
os.mkdir("dataset_generated/trains_shapes")

# for each trid_id, save all the corrisponding shapes in a file
for trip_id, grouped_data in tqdm(shapes_gtfs_merged.groupby("trip_id")):
    grouped_data[["shape_pt_lat", "shape_pt_lon"]].to_csv("dataset_generated/trains_shapes/{}.csv".format(trip_id), index=False)

100%|██████████| 13828/13828 [00:38<00:00, 361.56it/s]


In [114]:
# zip the folder with shutil
shutil.make_archive("dataset_generated/trains_shapes", 'zip', "dataset_generated/trains_shapes")

'/Users/giacomoorsi/DocsNotSynced/OpenRitardi/data/dataset_generated/trains_shapes.zip'